In [31]:
import pandas as pd

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.utilities import PythonREPL
# from langchain_community.chat_models import ChatLiteLLM
# from litellm import completion
from langchain_community.chat_models import ChatOllama

from langchain_core.runnables import chain

In [37]:
template = """
    Write some python  code to solve the user's problem. 
    Include print(result) at the end for user to check the result. 
    Return only python code and nothing else."""

prompt = ChatPromptTemplate.from_messages(
    [("system", template), ("human", "{question}")]
)

# model = ChatOpenAI(temperature=0, model_name="gpt-4")

model = ChatOllama(base_url="http://localhost:11434", model="codellama", streaming=False)

# model = ChatOllama(base_url="http://localhost:11434", model="mistral", streaming=False)

#  Python codes generation
chain_PythonCode = prompt | model | StrOutputParser()

# Python REPL (Read-Eval-Print Loop)
chain_PythonCodeRun = chain_PythonCode | PythonREPL().run


In [23]:
chain_PythonCode.invoke({"question": "How to plot line chart with dataframe df having columns of ['Timestamp', 'Col']"})

" To plot a line chart using a DataFrame `df` with columns named 'Timestamp' and 'Col', you can utilize the matplotlib library in Python. Here is the code:\n\n```python\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Assuming df is your DataFrame\nplt.figure(figsize=(12, 6))  # Set the figure size\n\nline = plt.plot(df['Timestamp'], df['Col'])[0]\n\nplt.xlabel('Timestamp')\nplt.ylabel('Col')\nplt.title('Line Chart')\nplt.xticks(rotation=45)  # Rotate x labels if needed\n\nplt.grid()\nplt.show()\n\nprint(result)  # result is an empty variable, you should replace it with the actual value you want to print out.\n```\n\nThis code will create a line chart using the 'Timestamp' column as the x-axis and the 'Col' column as the y-axis. The figure size is set to 12 inches by 6 inches. You can customize this according to your preference."

In [28]:
chain_PythonCodeRun.invoke({"question": "How to plot line chart with dataframe df having columns of ['Timestamp', 'Col']"})

"SyntaxError('invalid syntax', ('<string>', 1, 1, '```\\n', 1, 2))"

In [34]:
chain_PythonCodeRun.invoke({"question": "How much is 5 factorial?"})

"SyntaxError('invalid syntax', ('<string>', 1, 1, '```\\n', 1, 2))"

In [35]:
print(chain_PythonCode.invoke({"question": "How much is 5 factorial?"}))

```
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

result = factorial(5)
print(result)
```


In [39]:
str_code = chain_PythonCode.invoke({"question": "How much is 5 factorial?"})

In [45]:
str_code.replace("`", "'")

"'''\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\nresult = factorial(5)\nprint(result)\n'''"

In [42]:
type(str_code)

str

In [66]:
exec(str_code.replace("`", "'"))

In [68]:
str_code.replace("```", "'''")

"'''\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\nresult = factorial(5)\nprint(result)\n'''"

In [69]:
python_code = '''
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

result = factorial(5)
print(result)
'''


In [70]:
exec(python_code)


120


In [75]:
exec(
'''
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

result = factorial(5)
print(result)
'''
)

120


In [47]:
str_code

'```\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\nresult = factorial(5)\nprint(result)\n```'

In [49]:
print(str_code.replace("`", "'"))

'''
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

result = factorial(5)
print(result)
'''


In [51]:
exec(print(str_code))

```
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

result = factorial(5)
print(result)
```


TypeError: exec() arg 1 must be a string, bytes or code object

In [55]:
def execute_and_return(python_code):
    """Executes the provided Python code and returns the output value."""
    # Create an empty namespace to store the executed code's variables
    namespace = {}
    # Execute the Python code within the provided namespace
    exec(python_code, globals(), namespace)
    # Extract the result variable from the namespace
    result = namespace.get("result")
    # Return the extracted result value
    return result

In [58]:
aaa = execute_and_return(str_code.replace("```", "'''"))

In [59]:
aaa

In [54]:
print(str_code.replace("```", "'''"))

'''
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

result = factorial(5)
print(result)
'''


In [53]:
exec(str_code.replace("```", "'''"))

In [38]:
exec(chain_PythonCode.invoke({"question": "How much is 5 factorial?"}))

SyntaxError: invalid syntax (<string>, line 1)

In [2]:
df_data = pd.read_csv("/home/sprk/workspace/project/langchain/streamlit-app/multi-doc-rag/data/energy_production.csv")

In [3]:
df_data

,Year,Population(M),Coal,Oil,Gas,Nuclear
0,2000,282.17,6968,394,2179,2672
1,2001,285.08,6679,438,2274,2697
2,2002,287.80,6717,329,2441,2710
3,2003,290.33,6798,411,2292,2631
4,2004,293.05,6751,413,2475,2691
5,2005,295.75,6806,413,2618,2644
6,2006,298.59,6666,215,2782,2636
7,2007,301.58,6686,218,3018,2674
8,2008,304.38,6524,152,2939,2649
9,2009,307.01,5719,127,3034,2602


In [7]:
df_plot = pd.melt(df_data,
        id_vars = ['Year'],
        value_vars = ['Population(M)', 'Coal', 'Oil', 'Gas', 'Nuclear'],
        var_name = 'Metric',
        value_name = 'Value'
)

In [ ]:
template = """
    Write some python  code to solve the user's problem. 
    Include print(result) at the end for user to check the result. 
    Return only python code and nothing else."""

prompt = ChatPromptTemplate.from_messages(
    [("system", template), ("human", "{question}")]
)


In [32]:
@chain
def custom_chain(text):
    prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")
    
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = model.invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | model | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})

In [33]:
custom_chain.invoke("bears")

' The subject of this joke is "bear"'

In [ ]:
@chain
def custom_chain_plot(text, df):
    prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")
    
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = model.invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | model | StrOutputParser()
    # return chain2.invoke({"joke": parsed_output1})

    return chain2.invoke({"joke": parsed_output1})

In [ ]:
custom_chain_plot

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]